In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# magic command for matplotlib to enable printing directly in the jupyter notebook
%matplotlib inline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
#from sklearn.metrics import plot_confusion_matrix
#from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.utils import resample,shuffle
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score



In [ ]:
# PATHS

# get working directory
path_working = Path.cwd()

path_data = str(path_working) + "\\" + "data" + "\\"

#print(path_data)


In [ ]:
def create_dataframe (path):
    # Get the files from the path provided in the OP
    files = Path(path).glob('*.csv')  # .rglob to create iterator for subdirectories
    
    #Find all csv files in directory and concatenate them
    dfs = list()
    names = list()
    for file in files:
        data = pd.read_csv(file)
        # .stem is method for pathlib objects to get the filename without the extension
        # create another column in dataframe for filename
        data['file'] = file.stem
        names.append(file.stem)
        dfs.append(data)
        df = pd.concat(dfs, ignore_index=True)
    
    return df


In [ ]:
df_raw = create_dataframe(path_data)

In [ ]:
df_raw

In [ ]:
#drop last column taht shows csv filename source:
del df_raw[df_raw.columns[-1]]

df_raw

In [ ]:
print(df_raw.shape)

df_raw['marker'].value_counts()

### ---> unbalanced classes

In [ ]:
df_raw.info()

In [ ]:
# Look at datatypes of coloumns.
def show_dtypes(df):
    for index in range(len(df.dtypes)):
        print(f'{df.columns[index]} -> {df.dtypes[index]}' )
        
show_dtypes(df_raw)

In [ ]:
# .copy() is very important otherwise its just a pointer
df = df_raw.copy()

In [ ]:
def marker_to_numeric_triple(x):
        if x=='Attack': return 0
        if x=='Natural': return 1
        if x=='NoEvents': return 2
        
LABEL_MAP = {0 : "Attack", 1: "Natural", 2: "NoEvents"}

LABELNAMES = [LABEL_MAP[key] for key in LABEL_MAP]

df['marker'] = df['marker'].apply(marker_to_numeric_triple)
df

In [ ]:
# Check for infinite float entries
# "np.finfo(np.float64).max" returns the maximum finite value that can be represented by a 64-bit floating point number
# "df.values >= np.finfo(np.float64).max" returns a bollean array where "np.finfo(np.float64).max" is checked
# "np.where(df.values >= np.finfo(np.float64).max)" returns a array of indices where "df.values >= np.finfo(np.float64).max" is true

np.where(df.values >= np.finfo(np.float64).max)

### ----> we have infinite float entries

In [ ]:
# Check for NaN values
np.isnan(df.values.any())

### ----> we have no NaN values in the dataframe

In [ ]:
# Replace infinite updated data with nan

# "[np.inf, -np.inf]" specifies the values to be replaced
# "np.nan" specifies the value to replace the infinite values with.
# "inplace=True" modifies the Dataframe in place, so that the original df is updated the new values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN
df.dropna(inplace=True)

print(df.shape)

df['marker'].value_counts()

### ----> Due to cleaning up NaNs and infinites we lost about 6000 datapoints, which is not very harmful due to the fact that there are still 72000 rows

In [ ]:
y = df['marker']

X = df.drop(['marker'],axis =1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

In [ ]:
# define the model
xgb = xgb.XGBClassifier()

# define hyperparameters to tune
params = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 500, 1000],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.5, 0.8, 1],
    'colsample_bytree': [0.5, 0.8, 1]
}

In [ ]:
def evaluate_model(model, X_test, y_test, labelmap):
    """
    Evaluate a multiclass classification model using various metrics.
    
    Args:
        model (sklearn estimator): The model to evaluate.
        X_test (pandas dataframe): The test features.
        y_test (pandas series): The true labels
        labelmap (dict): The encoding of the names of the labels that are passed to report and matrix
        
    Returns:
        sklearn confusionmatrix
    """
    
    labelnames = [labelmap[key] for key in labelmap]
    pos_label = "Attack"
    
    # Predict the test labels using the trained model
    y_pred = model.predict(X_test)
    
    # map encoded labels to original labels
    y_true = [labelmap[i] for i in y_test]
    y_pred = [labelmap[i] for i in y_pred]
    
    # Compute the confusion matrix and print it
    confusionmatrix = confusion_matrix(y_true, y_pred, labels=labelnames)
    print("Confusion matrix:")
    print(confusionmatrix)
    
    # Compute the classification report and print it
    report = classification_report(y_true, y_pred, target_names=labelnames)
    print("Classification report:")
    print(report)
    
    return confusionmatrix, report

In [ ]:
# use GridSearchCV to find the best hyperparameters
grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='accuracy', cv=5, n_jobs=-1)
grid.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)

# print the best hyperparameters
print('Best hyperparameters:', grid.best_params_)

In [ ]:
cm, rp = evaluate_model(bst, X_test, y_test, LABEL_MAP)

In [ ]:
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELNAMES, yticklabels=LABELNAMES)

# UPSAMPLING

In [ ]:
df_minority_NoEvents = df[df['marker'] == 0]
df_minority_Natural = df[df['marker'] == 1]
df_not_minority = df[df['marker'] == 2] 

In [ ]:
#upsample the minority classes
df_minority_NoEvents_upsampled = resample(df_minority_NoEvents,random_state=42,n_samples=51445,replace=True)
df_minority_Natural_upsampled = resample(df_minority_Natural,random_state=42,n_samples=51445,replace=True)

In [ ]:
print(df_minority_NoEvents_upsampled.shape)
print(df_minority_Natural_upsampled.shape)

In [ ]:
#concatenate the upsampled dataframes
df_upsampled = pd.concat([df_minority_NoEvents_upsampled, df_minority_Natural_upsampled, df_not_minority ])
#df_upsampled

In [ ]:
y = df_upsampled['marker']

X = df_upsampled.drop(['marker'],axis =1)

In [ ]:
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

print("upsampled Train data:")
print(X_train.shape, y_train.shape)

print("untouched Test data:")
print(X_test.shape, y_test.shape)

In [ ]:
#RF with Upsampled Data
model_rf_upsampled = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
%time model_rf_upsampled.fit(X_train, y_train) 

In [ ]:
cm, rp = evaluate_model(model_rf_upsampled, X_test, y_test, LABEL_MAP)

In [ ]:
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELNAMES, yticklabels=LABELNAMES)